In [1]:
from llama_index import LLMPredictor, ServiceContext
# from llama_index import VectorStoreIndex
# from llama_index import SimpleDirectoryReader
from llama_index import Prompt
from llama_index import PromptHelper
from llama_index import StorageContext, load_index_from_storage
from llama_index.evaluation import ResponseEvaluator
from llama_index.llms import OpenAI
from llama_index.indices.postprocessor import SimilarityPostprocessor
from llama_index.indices.query.query_transform import HyDEQueryTransform
from llama_index.query_engine.transform_query_engine import TransformQueryEngine
from langchain.chat_models import ChatOpenAI
from IPython.display import Markdown, display

import environ
import openai

In [2]:
# For now I use my key
env = environ.Env()
environ.Env.read_env()
API_KEY = env("OPENAI_API_KEY")
openai.api_key = API_KEY

/home/daniele/Desktop/Projects/bureaubot/.venv_bureaubot/lib/python3.10/site-packages/environ/environ.py:639: UserWarning: /tmp/ipykernel_20332/.env doesn't exist - if you're not configuring your environment separately, create one.
  warnings.warn(


# LLM

In [3]:
system_prompt = (
    """
    You are an expert in the German administration system and your job is to answer technical questions.
    Assume that all questions are related to the the provided context.
    Keep your answers based on facts, do not hallucinate information.
    """
)

selected_llm = OpenAI(
    temperature=0,
    model_name="gpt-3.5-turbo",
    system_prompt=system_prompt
)

# Load vector store

In [4]:
embedding_path = "vector_db"
# rebuild storage context
storage_context_from_load = StorageContext.from_defaults(persist_dir=embedding_path)
# load index
index_loaded = load_index_from_storage(storage_context_from_load)

# Functions

In [5]:
def print_response_info(user_query, response):
    print(f"Prompt:\n{user_query}")
    display(Markdown(f"Response:<br/>{response.response}"))
    print(f"Sources:")
    for node in response.source_nodes:
        print(f'Page: {node.node.metadata["page_label"]}, Document: {node.node.metadata["file_name"]}, Cosine Similarity: {node.score:.2f}')

# Define query and prompt template, folders and top_k

In [6]:
# user_query = "Is the number of children taken into account to calculate the unemployment benefit?"
user_query = "How can I calculate how much money I will get as unemployment benefit?"
# user_query = "What do you know about the city of Berlin?" # check for generic responses from GPT knowledge base
# user_query = "Wann muss ich mich melden wenn ich Arbeitslosengeld beantragen will?"

# Define prompt
template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "Do not give me an answer if it is not mentioned in the context as a fact. \n"
    "Given this information, please provide me with an answer to the following question:\n{query_str}\n"
)
qa_template = Prompt(template)

folder_with_index = "vector_db"
number_top_results = 5 # Number of top results to return

# Query Engine

## Simple query engine

In [7]:
query_engine_baseline = index_loaded.as_query_engine(text_qa_template=qa_template, similarity_top_k=number_top_results)
response_baseline = query_engine_baseline.query(user_query)

print_response_info(user_query, response_baseline)

Prompt:
How can I calculate how much money I will get as unemployment benefit?


Response:<br/>To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:

1. Bemessungsentgelt: The amount of unemployment benefit is based on your average income in the last year before becoming unemployed. This is known as the "Bemessungsentgelt."

2. Lohnsteuerklasse: Your tax class also affects the calculation of unemployment benefit. The daily benefit rate is determined based on a standardized net income, taking into account the social security lump sum (20% of the Bemessungsentgelt), solidarity surcharge, and income tax, considering the tax class.

3. Number of children: If you have at least one child, the unemployment benefit amount may be increased to 67% of the standardized net income.

4. Other factors: There may be additional considerations or special regulations depending on your specific circumstances, such as reduced working hours due to childcare or caring for a family member.

To get an estimate of your unemployment benefit amount, you can use the self-calculation program provided by the employment agency. You can access this program on the website www.arbeitsagentur.de.

It's important to note that the specific calculation and final determination of your unemployment benefit will be provided in the approval notice from your employment agency, which will include the relevant calculation details.

Sources:
Page: 9, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: 0.82
Page: 37, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.82
Page: 5, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: 0.81
Page: 43, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.81
Page: 23, Document: dok_ba015377.pdf, Cosine Similarity: 0.81


In [8]:
for uwdfujsn in response_baseline.source_nodes:
    print(uwdfujsn.node.metadata)

{'page_label': '9', 'file_name': 'merkblatt-nebeneinkommen_ba015792.pdf'}
{'page_label': '37', 'file_name': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}
{'page_label': '5', 'file_name': 'merkblatt-nebeneinkommen_ba015792.pdf'}
{'page_label': '43', 'file_name': 'merkblatt-fuer-arbeitslose_ba036520.pdf'}
{'page_label': '23', 'file_name': 'dok_ba015377.pdf'}


In [9]:
response_text = response_baseline.response
response_metadata = dict()
response_metadata_message = f'There {len(response_baseline.metadata)} sources:'
for i, meta_data in enumerate(response_baseline.metadata):
    key_name = "ref_" + str(i)
    response_metadata[key_name] = {
        "page": response_baseline.metadata[meta_data]["page_label"],
        "document":response_baseline.metadata[meta_data]["file_name"]
    }
    response_metadata_message += "\n---" + key_name + f'Page {response_baseline.metadata[meta_data]["page_label"]} from file {response_baseline.metadata[meta_data]["file_name"]}'

In [10]:
# response_baseline

In [11]:
response_metadata

{'ref_0': {'page': '9', 'document': 'merkblatt-nebeneinkommen_ba015792.pdf'},
 'ref_1': {'page': '37',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_2': {'page': '5', 'document': 'merkblatt-nebeneinkommen_ba015792.pdf'},
 'ref_3': {'page': '43',
  'document': 'merkblatt-fuer-arbeitslose_ba036520.pdf'},
 'ref_4': {'page': '23', 'document': 'dok_ba015377.pdf'}}

In [12]:
print(response_metadata_message)

There 5 sources:
---ref_0Page 9 from file merkblatt-nebeneinkommen_ba015792.pdf
---ref_1Page 37 from file merkblatt-fuer-arbeitslose_ba036520.pdf
---ref_2Page 5 from file merkblatt-nebeneinkommen_ba015792.pdf
---ref_3Page 43 from file merkblatt-fuer-arbeitslose_ba036520.pdf
---ref_4Page 23 from file dok_ba015377.pdf


In [13]:
print(response_baseline.get_formatted_sources())

> Source (Doc id: 26a2e343-6e3f-4259-a610-4b7567ad302f): 9erhalten, werden mit Ihrem Nettobetrag berücksich -
tigt und nach Abzug eines Freibetrages von 4...

> Source (Doc id: e93526b3-c61a-48ee-9671-db13277ee364): 4. Die Höhe der Leistung
384. Die Höhe der Leistung
Für die Höhe des Arbeitslosengeldes sind von ...

> Source (Doc id: 0fa8296e-e554-44ab-b518-b12e076a1aca): Wie wird Nebeneinkommen 
angerechnet?
Wenn Sie ein monatliches Nebeneinkommen in gleich  
bleiben...

> Source (Doc id: 94e9bd6f-e3b8-4c10-869b-bc42cfb74683): 4. Die Höhe der Leistung
44erster Tag der Arbeitslosigkeit) bereits Arbeitslosen ­
geld bezogen, ...

> Source (Doc id: d0920db7-26ff-4398-8b30-c5885c17b25a): 233 Finanzen
3 Finanzen
3.1 Informationen zum Arbeitslosengeld
Wenn Sie einen beruflichen Wiedere...


## Cosine Similarity Postprocessor

In [14]:
for node in response_baseline.source_nodes:
    cosine_similarity = node.score
    print(f"Cosine Similarity: {cosine_similarity}")

Cosine Similarity: 0.8201131049473437
Cosine Similarity: 0.815204569991885
Cosine Similarity: 0.8108160176859768
Cosine Similarity: 0.8086722220407534
Cosine Similarity: 0.8081491623074246


In [15]:
postprocessor = SimilarityPostprocessor(similarity_cutoff=0.821)
postprocessed_nodes = postprocessor.postprocess_nodes(response_baseline.source_nodes)

for node in postprocessed_nodes:
    print(f'Page: {node.node.metadata["page_label"]}, Document: {node.node.metadata["file_name"]}, Cosine Similarity: {node.score:.2f}')

## SVM & linear regression & logistic regression

In [16]:
query_modes = [
    "svm",
    "linear_regression",
    "logistic_regression",
]

responses_modes = []

for query_mode in query_modes:
    query_engine_modes = index_loaded.as_query_engine(
        text_qa_template=qa_template,
        similarity_top_k=number_top_results,
        vector_store_query_mode=query_mode)

    response_modes = query_engine_modes.query(user_query)
    responses_modes.append(response_modes)

/home/daniele/Desktop/Projects/bureaubot/.venv_bureaubot/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/daniele/Desktop/Projects/bureaubot/.venv_bureaubot/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/daniele/Desktop/Projects/bureaubot/.venv_bureaubot/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(


In [17]:
for query_mode, r in zip(query_modes, responses_modes):
    print(f"Query mode: {query_mode}\n")
    print_response_info(user_query, r)
    print("="*90)

# Note that with SVM, the score is NOT cosine similarity but it's a score from the decision function from the SVM.
# In the decision function, each element represents whether a predicted sample for x (i.e., a chunk of text in the
# vector store) by the classifier lies to the right or left side of the Hyperplane and also how far from the Hyperplane.
# The smaller is the score (i.e., closer to zero from the negative side because we only have a single positive example,
# that is the query), the more similar is the x (i.e., the chunk of text) to the positive example (i.e., query).

Query mode: svm

Prompt:
How can I calculate how much money I will get as unemployment benefit?


Response:<br/>Based on the provided context information, there are two different scenarios mentioned for calculating the amount of unemployment benefit:

1. Nebeneinkommen (Side Income): If you have a monthly side income, the unemployment benefit will be reduced by the amount of the side income minus the exemption amount. The specific calculation can be found in the approval/changes notice (Bewilligungs-/Änderungsbescheid) provided by the employment agency.

2. Entlassungsentschädigung (Severance Pay): If you receive severance pay, a certain percentage of the pay will be considered when calculating the suspension period (Ruhenszeitraum) for unemployment benefit. The specific calculation can be found in the table mentioned in section 4.4 of the document.

It is important to note that these calculations may vary depending on individual circumstances and the specific regulations of the employment agency. It is recommended to consult with the responsible job center or employment agency for accurate information regarding the calculation of unemployment benefits in your specific situation.

Sources:
Page: 5, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.66
Page: 9, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.69
Page: 30, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.69
Page: 29, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.70
Page: 90, Document: merkblatt-algii_ba015397.pdf, Cosine Similarity: -0.71
Query mode: linear_regression

Prompt:
How can I calculate how much money I will get as unemployment benefit?


Response:<br/>Based on the provided context information, there are two different scenarios mentioned for calculating the amount of unemployment benefit:

1. Nebeneinkommen (Side Income): If you have a monthly side income, the unemployment benefit will be reduced by the amount of the income minus the exemption amount. The specific calculation can be found in the approval/changes notice provided by the employment agency.

2. Entlassungsentschädigung (Severance Pay): If you receive severance pay, a certain percentage of the pay will be considered when calculating the suspension period for unemployment benefits. The specific calculation can be found in the table mentioned in the document.

It is important to note that these calculations may vary depending on individual circumstances and the specific regulations of the employment agency. It is recommended to contact the responsible job center for accurate information regarding the calculation of unemployment benefits.

Sources:
Page: 5, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.66
Page: 9, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.69
Page: 30, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.69
Page: 29, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.70
Page: 90, Document: merkblatt-algii_ba015397.pdf, Cosine Similarity: -0.71
Query mode: logistic_regression

Prompt:
How can I calculate how much money I will get as unemployment benefit?


Response:<br/>Based on the provided context information, the calculation of unemployment benefits may vary depending on the specific circumstances and regulations in your country or region. It is recommended to consult the relevant government agency or unemployment office for accurate and up-to-date information on how to calculate unemployment benefits in your specific situation. They will be able to provide you with the necessary guidelines and requirements for determining the amount of money you will receive as unemployment benefits.

Sources:
Page: 5, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.66
Page: 9, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: -0.69
Page: 30, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.69
Page: 29, Document: merkblatt-17-entschaedigungen_ba015376.pdf, Cosine Similarity: -0.70
Page: 90, Document: merkblatt-algii_ba015397.pdf, Cosine Similarity: -0.71


## Hypothetical document embeddings (HyDE)

In [18]:
query_engine_hyde = index_loaded.as_query_engine(text_qa_template=qa_template, similarity_top_k=number_top_results)

# "HyDEQueryTransform" generates a hypothetical document and use it for embedding lookup.
hyde = HyDEQueryTransform(include_original=True)
hyde_query_engine = TransformQueryEngine(query_engine_hyde, hyde)
response_hyde = hyde_query_engine.query(user_query)

print_response_info(user_query, response_hyde)

Prompt:
How can I calculate how much money I will get as unemployment benefit?


Response:<br/>To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:

1. Bemessungsentgelt (Basis for Calculation): The amount of unemployment benefit is based on your average income during a specific period before becoming unemployed. This is known as the Bemessungsentgelt.

2. Leistungssatz (Benefit Rate): The benefit rate is determined based on your personal circumstances, such as whether you have children or not. The standard benefit rate is 60% of the Bemessungsentgelt, but it can increase to 67% if you have at least one child.

3. Lohnsteuerklasse (Tax Class): Your tax class also affects the calculation of unemployment benefit. The specific impact depends on the tax class you had at the beginning of the year when the entitlement to benefits arose.

4. Anwartschaftszeit (Qualifying Period): To be eligible for unemployment benefit, you must have fulfilled the Anwartschaftszeit, which is the minimum period of employment required.

To calculate the exact amount of unemployment benefit, you can use the self-calculation program provided by the employment agency. You can access this program on the website www.arbeitsagentur.de.

Please note that the specific calculation may vary depending on your individual circumstances and the regulations of your country. It is recommended to consult with your local job center or employment agency for accurate information regarding your entitlement to unemployment benefits.

Sources:
Page: 9, Document: merkblatt-nebeneinkommen_ba015792.pdf, Cosine Similarity: 0.84
Page: 37, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.84
Page: 23, Document: dok_ba015377.pdf, Cosine Similarity: 0.84
Page: 43, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.83
Page: 39, Document: merkblatt-fuer-arbeitslose_ba036520.pdf, Cosine Similarity: 0.83


In [19]:
hyde

In [20]:
#In this example, HyDE improves output quality significantly, by hallucinating
# accurately, and thus improving the embedding quality, and final output.
query_bundle = hyde(user_query)
hyde_doc = query_bundle.embedding_strs[0]

print(f"Prompt:\n{user_query}\n\nResponse:")
display(Markdown(f"{hyde_doc}"))

Prompt:
How can I calculate how much money I will get as unemployment benefit?

Response:


To calculate how much money you will receive as unemployment benefits, you need to consider several key factors. Firstly, you should determine your base period, which is typically the first four of the last five completed calendar quarters before you filed your claim. This period is used to calculate your earnings during that time.

Next, you need to gather information about your earnings during the base period. This includes wages from all employers you worked for during that time, including part-time and temporary jobs. It's important to note that certain types of income, such as self-employment earnings or income from rental properties, may not be considered for unemployment benefits.

Once you have gathered your earnings information, you can calculate your average weekly wage (AWW). To do this, add up your total earnings during the base period and divide it by the number of weeks in that period. This will give you an estimate of your AWW.

After determining your AWW, you can then calculate your weekly benefit amount (WBA). This is usually a percentage of your AWW, which varies depending on the state you reside in. Each state has its own formula or table to determine the percentage, so it's important to consult your state's unemployment agency or website for the specific calculation.

Additionally, there may be a maximum weekly benefit amount (MBA) set by your state. If your calculated WBA exceeds the MBA, your benefit amount will be capped at the maximum limit.

It's important to note that unemployment benefits are subject to federal and state taxes. Depending on your state, taxes may be automatically deducted from your benefit payments, or you may be responsible for reporting and paying taxes on your own.

Lastly, keep in mind that unemployment benefits are typically provided for a limited duration, such as 26 weeks. However, during times of economic downturn or other exceptional circumstances, extended benefits may be available.

In summary, to calculate your unemployment benefits, determine your base period, gather earnings information, calculate your AWW, determine the percentage for your WBA based on your state's formula or table, consider the MBA if applicable, and be aware of tax obligations. Remember to consult your state's unemployment agency or website for specific guidelines and calculations.

In [21]:
hyde_2 = HyDEQueryTransform(include_original=True)
query_bundle_2 = hyde_2(user_query)
hyde_doc_2 = query_bundle_2.embedding_strs[0]

print(f"Prompt:\n{user_query}\n\nResponse:")
display(Markdown(f"{hyde_doc_2}"))

Prompt:
How can I calculate how much money I will get as unemployment benefit?

Response:


To calculate how much money you will receive as unemployment benefits, there are several key details you need to consider. Firstly, you will need to determine your base period, which is typically the first four out of the last five completed calendar quarters before you filed for unemployment. This period is used to calculate your earnings during that time.

Next, you will need to gather information about your earnings during the base period. This includes wages from all employers you worked for during that time, including part-time and temporary jobs. It is important to note that certain types of income, such as self-employment earnings or income from rental properties, may not be considered for unemployment benefit calculations.

Once you have gathered your earnings information, you can proceed to calculate your average weekly wage (AWW). This is done by dividing your total earnings during the base period by the number of weeks you worked. It is crucial to accurately report your earnings to ensure an accurate calculation.

After determining your AWW, you can then calculate your weekly benefit amount (WBA). This is usually a percentage of your AWW, which varies depending on the state you reside in. Each state has its own formula or table to determine the percentage, so it is essential to consult your state's unemployment agency or website for specific information.

Additionally, there may be a maximum weekly benefit amount (MBA) set by your state. If your calculated WBA exceeds the MBA, your benefit amount will be capped at the maximum limit. It is important to be aware of this limit to manage your expectations.

Finally, it is crucial to understand that unemployment benefits are subject to federal and state taxes. Depending on your state's regulations, taxes may be automatically deducted from your benefit payments or you may be responsible for reporting and paying taxes on your own.

In summary, to calculate your unemployment benefits, you need to determine your base period, gather your earnings information, calculate your average weekly wage, determine the percentage for your weekly benefit amount, consider any maximum weekly benefit amount, and be aware of tax implications. By understanding these key details and consulting your state's unemployment agency, you can accurately estimate how much money you will receive as unemployment benefits.

# Chat Engine

In [22]:
# Configure prompt parameters and initialise helper
max_input_size = 4096
num_output = 256
max_chunk_overlap = 0.2
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

# system_prompt = (
#     """
#     You are an expert on the German administration system and your job is to answer technical questions.
#     Assume that all questions are related to the the provided context.
#     Keep your answers based on facts, do not hallucinate information.
#     """
# )
# 
# llm= LLMPredictor(llm=OpenAI(
#     temperature=0,
#     model_name="gpt-3.5-turbo",
#     system_prompt=system_prompt
# ))
llm = LLMPredictor(selected_llm)
service_context = ServiceContext.from_defaults(llm_predictor=llm, prompt_helper=prompt_helper)

chat_engine = index_loaded.as_chat_engine(
    # service_context=service_context,
    chat_mode="context",
    # verbose=True
    similarity_top_k=3
)

In [23]:
response_chat = chat_engine.chat(user_query)

In [24]:
# response_chat

In [25]:
print(f"There {len(response_chat.source_nodes)} sources.")
for source_node in response_chat.source_nodes:
    print(f'Page {source_node.node.metadata["page_label"]} from file {source_node.node.metadata["file_name"]}')

There 3 sources.
Page 9 from file merkblatt-nebeneinkommen_ba015792.pdf
Page 37 from file merkblatt-fuer-arbeitslose_ba036520.pdf
Page 5 from file merkblatt-nebeneinkommen_ba015792.pdf


In [26]:
response_chat.response

"To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:\n\n1. Bemessungsentgelt (Basis for Calculation): This is the average gross income you earned in the last year before becoming unemployed. It is used to determine the amount of your benefit.\n\n2. Lohnsteuerklasse (Tax Class): Your tax class affects the deductions made from your benefit. Different tax classes have different deduction rates.\n\n3. Number of Children: If you have children, they may entitle you to an increased benefit rate.\n\nOnce you have these details, you can use the following steps to calculate your unemployment benefit:\n\n1. Determine your Bemessungsentgelt: Add up your gross income from the last year and divide it by the number of days you worked during that period.\n\n2. Calculate your daily benefit rate: Divide your Bemessungsentgelt by 365 (or 360 if your benefit is paid monthly) to get your daily benefit rate.\n\n3. Apply the tax class deduction: M

In [27]:
len(response_chat.source_nodes)

3

In [28]:
response_chat.source_nodes[0]

NodeWithScore(node=TextNode(id_='26a2e343-6e3f-4259-a610-4b7567ad302f', embedding=None, metadata={'page_label': '9', 'file_name': 'merkblatt-nebeneinkommen_ba015792.pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='e864b8b6-a752-4fe3-8bc8-9e112d83bfc7', node_type=None, metadata={'page_label': '9', 'file_name': 'merkblatt-nebeneinkommen_ba015792.pdf'}, hash='76c187b531eba95cd2f7d95a6eb6a6eff2fde5336100e8cae1869661bc1e520a')}, hash='76c187b531eba95cd2f7d95a6eb6a6eff2fde5336100e8cae1869661bc1e520a', text='9erhalten, werden mit Ihrem Nettobetrag berücksich -\ntigt und nach Abzug eines Freibetrages von 400 Euro \nmonatlich auf das Arbeitslosengeld angerechnet.\nBEISPIEL 4\nWegen der Teilnahme an einer Weiterbildung \nerhalten Sie von Ihrem Arbeitgeber eine monat -\nliche Vergütung von 500 Euro netto. Nach Ab -\nzug des Freibetrages von 400 Euro monatlich \nwerden 100 Euro monatlich auf das Arbeits-\

In [29]:
response_chat.source_nodes[0].node.metadata["page_label"]

'9'

# Compare responses

In [30]:
methods_list = ["Baseline", "SVM", "Linear regression", "Logistic regression", "HyDE", "HyDE doc", "Chat"]
responses_list = [
    response_baseline.response, responses_modes[0].response, responses_modes[1].response,
    responses_modes[2].response, response_hyde.response, hyde_doc, response_chat.response]

for m, r in zip(methods_list, responses_list):
    display(Markdown(f"{m}:<br/>{r}"))
    print("="*80)


Baseline:<br/>To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:

1. Bemessungsentgelt: The amount of unemployment benefit is based on your average income in the last year before becoming unemployed. This is known as the "Bemessungsentgelt."

2. Lohnsteuerklasse: Your tax class also affects the calculation of unemployment benefit. The daily benefit rate is determined based on a standardized net income, taking into account the social security lump sum (20% of the Bemessungsentgelt), solidarity surcharge, and income tax, considering the tax class.

3. Number of children: If you have at least one child, the unemployment benefit amount may be increased to 67% of the standardized net income.

4. Other factors: There may be additional considerations or special regulations depending on your specific circumstances, such as reduced working hours due to childcare or caring for a family member.

To get an estimate of your unemployment benefit amount, you can use the self-calculation program provided by the employment agency. You can access this program on the website www.arbeitsagentur.de.

It's important to note that the specific calculation and final determination of your unemployment benefit will be provided in the approval notice from your employment agency, which will include the relevant calculation details.

SVM:<br/>Based on the provided context information, there are two different scenarios mentioned for calculating the amount of unemployment benefit:

1. Nebeneinkommen (Side Income): If you have a monthly side income, the unemployment benefit will be reduced by the amount of the side income minus the exemption amount. The specific calculation can be found in the approval/changes notice (Bewilligungs-/Änderungsbescheid) provided by the employment agency.

2. Entlassungsentschädigung (Severance Pay): If you receive severance pay, a certain percentage of the pay will be considered when calculating the suspension period (Ruhenszeitraum) for unemployment benefit. The specific calculation can be found in the table mentioned in section 4.4 of the document.

It is important to note that these calculations may vary depending on individual circumstances and the specific regulations of the employment agency. It is recommended to consult with the responsible job center or employment agency for accurate information regarding the calculation of unemployment benefits in your specific situation.

Linear regression:<br/>Based on the provided context information, there are two different scenarios mentioned for calculating the amount of unemployment benefit:

1. Nebeneinkommen (Side Income): If you have a monthly side income, the unemployment benefit will be reduced by the amount of the income minus the exemption amount. The specific calculation can be found in the approval/changes notice provided by the employment agency.

2. Entlassungsentschädigung (Severance Pay): If you receive severance pay, a certain percentage of the pay will be considered when calculating the suspension period for unemployment benefits. The specific calculation can be found in the table mentioned in the document.

It is important to note that these calculations may vary depending on individual circumstances and the specific regulations of the employment agency. It is recommended to contact the responsible job center for accurate information regarding the calculation of unemployment benefits.

Logistic regression:<br/>Based on the provided context information, the calculation of unemployment benefits may vary depending on the specific circumstances and regulations in your country or region. It is recommended to consult the relevant government agency or unemployment office for accurate and up-to-date information on how to calculate unemployment benefits in your specific situation. They will be able to provide you with the necessary guidelines and requirements for determining the amount of money you will receive as unemployment benefits.

HyDE:<br/>To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:

1. Bemessungsentgelt (Basis for Calculation): The amount of unemployment benefit is based on your average income during a specific period before becoming unemployed. This is known as the Bemessungsentgelt.

2. Leistungssatz (Benefit Rate): The benefit rate is determined based on your personal circumstances, such as whether you have children or not. The standard benefit rate is 60% of the Bemessungsentgelt, but it can increase to 67% if you have at least one child.

3. Lohnsteuerklasse (Tax Class): Your tax class also affects the calculation of unemployment benefit. The specific impact depends on the tax class you had at the beginning of the year when the entitlement to benefits arose.

4. Anwartschaftszeit (Qualifying Period): To be eligible for unemployment benefit, you must have fulfilled the Anwartschaftszeit, which is the minimum period of employment required.

To calculate the exact amount of unemployment benefit, you can use the self-calculation program provided by the employment agency. You can access this program on the website www.arbeitsagentur.de.

Please note that the specific calculation may vary depending on your individual circumstances and the regulations of your country. It is recommended to consult with your local job center or employment agency for accurate information regarding your entitlement to unemployment benefits.

HyDE doc:<br/>To calculate how much money you will receive as unemployment benefits, you need to consider several key factors. Firstly, you should determine your base period, which is typically the first four of the last five completed calendar quarters before you filed your claim. This period is used to calculate your earnings during that time.

Next, you need to gather information about your earnings during the base period. This includes wages from all employers you worked for during that time, including part-time and temporary jobs. It's important to note that certain types of income, such as self-employment earnings or income from rental properties, may not be considered for unemployment benefits.

Once you have gathered your earnings information, you can calculate your average weekly wage (AWW). To do this, add up your total earnings during the base period and divide it by the number of weeks in that period. This will give you an estimate of your AWW.

After determining your AWW, you can then calculate your weekly benefit amount (WBA). This is usually a percentage of your AWW, which varies depending on the state you reside in. Each state has its own formula or table to determine the percentage, so it's important to consult your state's unemployment agency or website for the specific calculation.

Additionally, there may be a maximum weekly benefit amount (MBA) set by your state. If your calculated WBA exceeds the MBA, your benefit amount will be capped at the maximum limit.

It's important to note that unemployment benefits are subject to federal and state taxes. Depending on your state, taxes may be automatically deducted from your benefit payments, or you may be responsible for reporting and paying taxes on your own.

Lastly, keep in mind that unemployment benefits are typically provided for a limited duration, such as 26 weeks. However, during times of economic downturn or other exceptional circumstances, extended benefits may be available.

In summary, to calculate your unemployment benefits, determine your base period, gather earnings information, calculate your AWW, determine the percentage for your WBA based on your state's formula or table, consider the MBA if applicable, and be aware of tax obligations. Remember to consult your state's unemployment agency or website for specific guidelines and calculations.

Chat:<br/>To calculate how much money you will receive as unemployment benefit, you need to consider the following factors:

1. Bemessungsentgelt (Basis for Calculation): This is the average gross income you earned in the last year before becoming unemployed. It is used to determine the amount of your benefit.

2. Lohnsteuerklasse (Tax Class): Your tax class affects the deductions made from your benefit. Different tax classes have different deduction rates.

3. Number of Children: If you have children, they may entitle you to an increased benefit rate.

Once you have these details, you can use the following steps to calculate your unemployment benefit:

1. Determine your Bemessungsentgelt: Add up your gross income from the last year and divide it by the number of days you worked during that period.

2. Calculate your daily benefit rate: Divide your Bemessungsentgelt by 365 (or 360 if your benefit is paid monthly) to get your daily benefit rate.

3. Apply the tax class deduction: Multiply your daily benefit rate by the deduction rate associated with your tax class. This will give you the net daily benefit rate.

4. Determine the number of days you will receive benefits: If you will receive benefits for a full calendar month, assume 30 days. If it is a partial month, calculate the exact number of days.

5. Multiply the net daily benefit rate by the number of days: Multiply your net daily benefit rate by the number of days you will receive benefits to calculate the total amount you will receive.

It's important to note that these calculations may vary depending on the specific regulations and rules of your country or state. It is recommended to consult with your local unemployment agency or visit their website for accurate and up-to-date information on calculating unemployment benefits.